In [1]:
import os
# import pandas as pd 
import polars as pl
# import scanpy as sc
import numpy as np
import typing as tp
import re
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier as KNN
# from sklearn.cross_decomposition import PLSCanonical, PLSRegression, CCA
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import cross_validate, cross_val_score
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
%matplotlib inline
import os, shutil, glob
import seaborn as sns; sns.set_style("white")
import umap as umap
import hdbscan as hdb

# import plotnine as gg
# from cytominer_eval import evaluate

os.getcwd()

2024-08-22 09:45:45.365753: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


'/scratch2-shared/david/colopaint3D/python'

In [4]:
figformat = 'png'
dpi = 300
statarg = 'single'
OutputDir = f'./data/2_PCAUMAP'
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)
FeatureDir = './data/1_FeaturesImages'


In [32]:
filenames = ['HCT116_MeanAgg_medianmad']
statmets = ['SingleCell', 'Aggregates']

dropcols = ['Nuclei_Location_Center_X',
            'Nuclei_Location_Center_Y',
            'Location',
            'ImageNumber_',
            'Parent',
            'Children',
            '_ObjectNumber',
            '_Object_Number',
            '_Y',
            '_X' 
            ]


cons = ['pos_con', 'neg_con']
float_columns=[pl.col(pl.Float32),pl.col(pl.Float64)]

##### Correlation contains some kind of well information
# feature_groups = ['AreaShape'
#                   , 'Correlation'
#                   , 'Granularity'
#                   , 'Intensity'
#                   , 'Neighbors'
#                   , 'RadialDistribution'
#                   ]
# real_names = [x for x in training_data.columns if x.split('_')[0] in feature_groups]
# training_data = training_data[real_names]

In [33]:
def readData(filename, statmet='SingleCell'):
    df = pl.read_parquet(f'{FeatureDir}/{statmet}/{filename}.parquet')
    ##This here is important!!!!!!
    # df=df.select([c for c in df.columns if not is_meta_column(c)])
    # dataNpy = df.to_numpy(df.select(float_columns))
    ##
    onehot_list = df.select(['Metadata_cmpd_pathway'])['Metadata_cmpd_pathway'].unique().to_list()
    onehot_mapping = {name: i for i, name in enumerate(onehot_list)}
    df = df.with_columns(df['Metadata_cmpd_pathway'].map_elements(lambda name: oneHot(name, onehot_mapping)).alias('Metadata_cmpd_onehot'))
    return df

# this is code from Dan
def is_meta_column(
    c:str,
    allowlist:tp.List[str]=["Metadata_Well","Metadata_Barcode","Metadata_AcqID","Metadata_Site"],
    denylist:tp.List[str]=[],
)->bool:
    """
        allowlist:
            the function will return False for these, no matter if they are metadata or not
        denylist:
            the function will return True for these, no matter if they are metadata or not
    """
    if c in allowlist:
        return False
    if c in denylist:
        return True
    for ex in '''
        Metadata
        ^Count
        ImageNumber
        Object
        Parent
        Children
        Plate
        Well
        Location
        _[XYZ]_
        _[XYZ]$
        BoundingBox
        Phase
        Orientation
        Angle
        Scale
        Scaling
        Width
        Height
        Group
        FileName
        PathName
        URL
        Execution
        ModuleError
        LargeBrightArtefact
        MD5Digest
        RadialDistribution_Frac
        Intensity_
    '''.split():
        if re.search(ex, c):
            return True
    return False


def oneHot(row, mapping):
    return mapping.get(row, -1)

def makePCA(df, name='', statmet='SingleCell' , n_components=2):
    dataN=df.select([c for c in df.columns if not is_meta_column(c)]).select(float_columns).to_numpy()
    pca_model = PCA(n_components=2)
    pca_model = pca_model.fit(dataN)
    pcaOut = pca_model.transform(dataN)
    df = df.with_columns([
    pl.Series('pc1', pcaOut[:, 0]),  
    pl.Series('pc2', pcaOut[:, 1])   
    ])

    hue = df['Metadata_cmpd_moa_group'].to_list()
    cmap = sns.color_palette("husl", n_colors=len(df['Metadata_cmpd_moa_group'].unique().to_list()))

    fig = plt.figure(
    # figsize=[14, 5]
    )
    ax = fig.add_subplot(111)
    ax.set_xlabel('PC 1', fontsize = 10)
    ax.set_ylabel('PC 2', fontsize = 10)
    ax.spines['top'].set_color('w')
    ax.spines['right'].set_color('w')
    ax.spines['left'].set_color('grey')
    ax.spines['bottom'].set_color('grey')
    sns.scatterplot(x=df['pc1'].to_list(),
                    y=df['pc2'].to_list(),
                    palette=cmap, hue=hue,
                    marker='.',
                    ).set(title=f'PCA {name} All'
                )
    ax.set_facecolor('w')
    ax.get_legend().remove()
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.show()
    plt.close()
    return df

def makeUMAP(df, name='', statmet='SingleCell' , nn = 200, is_supervised=True, n_components=100, min_dist=0.2, spread= 5, n_epochs=None, metric='cosine', use_pca=True):
    dataN=df.select([c for c in df.columns if not is_meta_column(c)]).select(float_columns).to_numpy()
    umap_model = umap.UMAP(n_neighbors=nn
                        , min_dist=min_dist
                        , spread= spread
                        , n_epochs=n_epochs
                        , metric=metric
                        , n_jobs=-1
                        )
    if use_pca:
        pca_model = PCA(n_components=n_components)
        pca_model = pca_model.fit(dataN)
        dataN = pca_model.transform(dataN)
        
    if is_supervised:
        umapOut = umap_model.fit_transform(dataN, y=df['Metadata_cmpd_one_hot'].to_list())
        isSup = 'Supervised'
    else:
        umapOut = umap_model.fit_transform(dataN)
        isSup = 'Unsupervised'
    df = df.with_columns([
    pl.Series('umap1', umapOut[:, 0]),  
    pl.Series('umap2', umapOut[:, 1])   
    ])

    hue = df['Metadata_cmpd_moa_group'].to_list()
    cmap = sns.color_palette("husl", n_colors=len(df['Metadata_cmpd_moa_group'].unique().to_list()))

    fig = plt.figure(
    # figsize=[14, 5]
    )
    ax = fig.add_subplot(111)
    ax.set_xlabel('UMAP 1', fontsize = 10)
    ax.set_ylabel('UMAP 2', fontsize = 10)
    ax.spines['top'].set_color('w')
    ax.spines['right'].set_color('w')
    ax.spines['left'].set_color('grey')
    ax.spines['bottom'].set_color('grey')
    sns.scatterplot(x=df['umap1'].to_list(),
                    y=df['umap2'].to_list(),
                    palette=cmap, hue=hue,
                    marker='.',
                    ).set(title=f'umap {name} All'
                )
    ax.set_facecolor('w')
    ax.get_legend().remove()
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    # if not os.path.exists(f'{OutputDir}/{statmet}'):
    #     os.makedirs(f'{OutputDir}/{statmet}')
    # plt.savefig(f'{OutputDir}/{statmet}/{name}_umap{nn}nn_{isSup}.png')
    plt.show()
    plt.close()
    return df


In [39]:
df = pl.read_parquet('data/1_FeaturesImages/Aggregates/HCT116_MeanAgg_medianmad.parquet')

In [35]:
df.head()

Metadata_PlateWell,Metadata_Barcode,Metadata_Well,Nuclei_RadialDistribution_ZernikeMagnitude_CONC_7_7,Cells_RadialDistribution_ZernikePhase_MITO_4_2,Nuclei_RadialDistribution_ZernikeMagnitude_PHAandWGA_9_1,Cytoplasm_Correlation_Correlation_PHAandWGA_SYTO,Cytoplasm_RadialDistribution_ZernikeMagnitude_HOECHST_4_4,Nuclei_Location_MaxIntensity_X_MITO,Cells_Granularity_13_MITO,Cells_RadialDistribution_ZernikePhase_CONC_6_0,Cytoplasm_RadialDistribution_ZernikePhase_MITO_2_2,Nuclei_RadialDistribution_RadialCV_MITO_1of4,Cells_RadialDistribution_ZernikeMagnitude_CONC_1_1,Nuclei_Correlation_RWC_SYTO_HOECHST,Nuclei_RadialDistribution_ZernikeMagnitude_SYTO_9_7,Cells_Intensity_MaxIntensityEdge_SYTO,Nuclei_Correlation_Manders_HOECHST_SYTO,Cytoplasm_RadialDistribution_ZernikePhase_CONC_8_4,Cytoplasm_Intensity_IntegratedIntensityEdge_PHAandWGA,Nuclei_Correlation_Overlap_CONC_MITO,Cytoplasm_RadialDistribution_ZernikePhase_CONC_8_2,Cells_AreaShape_EquivalentDiameter,Cells_RadialDistribution_ZernikePhase_CONC_5_1,Nuclei_RadialDistribution_ZernikePhase_HOECHST_9_3,Nuclei_RadialDistribution_ZernikeMagnitude_MITO_7_1,Nuclei_RadialDistribution_ZernikeMagnitude_PHAandWGA_9_3,Cells_RadialDistribution_ZernikeMagnitude_MITO_5_5,Cytoplasm_Granularity_3_HOECHST,Cells_RadialDistribution_ZernikePhase_MITO_9_9,Nuclei_RadialDistribution_ZernikePhase_SYTO_6_6,Nuclei_RadialDistribution_MeanFrac_MITO_2of4,Nuclei_Correlation_Manders_MITO_PHAandWGA,Cells_RadialDistribution_ZernikePhase_CONC_2_0,Cells_Correlation_Costes_CONC_PHAandWGA,Cytoplasm_RadialDistribution_ZernikeMagnitude_HOECHST_9_3,Nuclei_Correlation_RWC_MITO_HOECHST,…,Metadata_cmpd_cmpdname,Cytoplasm_PathName_SYTO,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_PathName_MITO,Metadata_cmpd_plate_well,Cytoplasm_Neighbors_SecondClosestObjectNumber_Adjacent,Metadata_cmpd_target_type,Nuclei_PathName_SYTO,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_EulerNumber,Cells_ImageNumber,Cells_PathName_SYTO,Cytoplasm_FileName_HOECHST,Cells_FileName_HOECHST,Nuclei_FileName_SYTO,Cytoplasm_AreaShape_Area,Cytoplasm_Children_cytoplasm_Count,Cytoplasm_Metadata_AcqID,Nuclei_FileName_PHAandWGA,Cytoplasm_PathName_HOECHST,Cells_FileName_PHAandWGA,Cytoplasm_Children_nuclei_Count,Metadata_cmpd_image_id,Cytoplasm_FileName_SYTO,Cytoplasm_Location_Center_Z,Metadata_cmpd_pert_type,Cytoplasm_PathName_CONC,Cells_Parent_cells,Cytoplasm_ObjectNumber,Cells_FileName_SYTO,Cytoplasm_ImageNumber,Cytoplasm_AreaShape_ConvexArea,Metadata_cmpd_cp_id,Metadata_cmpd_pathway,Cytoplasm_FileName_MITO,Cytoplasm_FileName_PHAandWGA
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,str,str,i32,str,str,i32,str,str,i32,i32,i32,i32,str,str,str,str,i32,i32,i32,str,str,str,i32,i64,str,i32,str,str,i32,i32,str,i32,i32,i64,str,str,str
"""PB000137_F08""","""PB000137""","""F08""",1.591848,0.020889,0.737056,-0.74838,0.609674,0.396334,11.070058,158.059344,0.071332,0.745752,1.353995,0.484623,0.885146,0.562251,0.0,-0.025716,0.646754,-1.222309,0.001439,-0.297185,-0.002363,0.005911,1.595439,0.804077,1.665911,1.511012,-0.04805,-0.007227,-6.420777,0.0,-137.158935,0.0,0.565479,-0.312143,…,"""Gefitinib (ZD1839)""","""/share/mikro2/nikon/spher-colo…",595,"""/share/mikro2/nikon/spher-colo…","""PB000137_F08""",4,"""Targeted""",null,677,5904,1,1,"""/share/mikro2/nikon/spher-colo…","""Well-F08-z6-HOECHST.ome.tiff""","""Well-F08-z6-HOECHST.ome.tiff""",null,3395,1,4185,null,"""/share/mikro2/nikon/spher-colo…","""Well-F08-z6-PHAandWGA.ome.tiff""",1,4185,"""Well-F08-z6-SYTO.ome.tiff""",0,"""trt""","""/share/mikro2/nikon/spher-colo…",1,1,"""Well-F08-z6-SYTO.ome.tiff""",1,3640,5532,"""Protein Tyrosine Kinase""","""Well-F08-z6-MITO.ome.tiff""","""Well-F08-z6-PHAandWGA.ome.tiff"""
"""PB000139_E08""","""PB000139""","""E08""",0.163568,-0.372447,-0.817861,-0.702877,-0.831863,0.144748,35.820364,173.228011,-0.101273,1.780737,-0.002377,1.15503

In [38]:
df.select(pl.col('Metadata_cmpd_pathway')).to_series().unique().to_list()

['Apoptosis',
 'Cell Cycle',
 'Cytoskeletal Signaling',
 'dmso',
 'water',
 'fenb',
 'etop',
 'stau',
 'MAPK',
 'TGF-beta/Smad',
 'JAK/STAT',
 'PI3K/Akt/mTOR',
 'Others',
 'Angiogenesis',
 'DNA Damage',
 'Proteases',
 'Stem Cells & Wnt',
 'Epigenetics',
 'sorb',
 'Protein Tyrosine Kinase']

In [41]:
filename = filenames[0]
# df = pl.read_parquet(f'{FeatureDir}/SingleCell/{filename}.parquet')
df = readData(filename, statmet='Aggregates')
# df['Metadata_cmpd_pathway'].unique().to_list()

/tmp/ipykernel_140683/1043507530.py:9: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(df['Metadata_cmpd_pathway'].map_elements(lambda name: oneHot(name, onehot_mapping)).alias('Metadata_cmpd_onehot'))


In [42]:
df.head()

Metadata_PlateWell,Metadata_Barcode,Metadata_Well,Nuclei_RadialDistribution_ZernikeMagnitude_CONC_7_7,Cells_RadialDistribution_ZernikePhase_MITO_4_2,Nuclei_RadialDistribution_ZernikeMagnitude_PHAandWGA_9_1,Cytoplasm_Correlation_Correlation_PHAandWGA_SYTO,Cytoplasm_RadialDistribution_ZernikeMagnitude_HOECHST_4_4,Nuclei_Location_MaxIntensity_X_MITO,Cells_Granularity_13_MITO,Cells_RadialDistribution_ZernikePhase_CONC_6_0,Cytoplasm_RadialDistribution_ZernikePhase_MITO_2_2,Nuclei_RadialDistribution_RadialCV_MITO_1of4,Cells_RadialDistribution_ZernikeMagnitude_CONC_1_1,Nuclei_Correlation_RWC_SYTO_HOECHST,Nuclei_RadialDistribution_ZernikeMagnitude_SYTO_9_7,Cells_Intensity_MaxIntensityEdge_SYTO,Nuclei_Correlation_Manders_HOECHST_SYTO,Cytoplasm_RadialDistribution_ZernikePhase_CONC_8_4,Cytoplasm_Intensity_IntegratedIntensityEdge_PHAandWGA,Nuclei_Correlation_Overlap_CONC_MITO,Cytoplasm_RadialDistribution_ZernikePhase_CONC_8_2,Cells_AreaShape_EquivalentDiameter,Cells_RadialDistribution_ZernikePhase_CONC_5_1,Nuclei_RadialDistribution_ZernikePhase_HOECHST_9_3,Nuclei_RadialDistribution_ZernikeMagnitude_MITO_7_1,Nuclei_RadialDistribution_ZernikeMagnitude_PHAandWGA_9_3,Cells_RadialDistribution_ZernikeMagnitude_MITO_5_5,Cytoplasm_Granularity_3_HOECHST,Cells_RadialDistribution_ZernikePhase_MITO_9_9,Nuclei_RadialDistribution_ZernikePhase_SYTO_6_6,Nuclei_RadialDistribution_MeanFrac_MITO_2of4,Nuclei_Correlation_Manders_MITO_PHAandWGA,Cells_RadialDistribution_ZernikePhase_CONC_2_0,Cells_Correlation_Costes_CONC_PHAandWGA,Cytoplasm_RadialDistribution_ZernikeMagnitude_HOECHST_9_3,Nuclei_Correlation_RWC_MITO_HOECHST,…,Cytoplasm_PathName_SYTO,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_PathName_MITO,Metadata_cmpd_plate_well,Cytoplasm_Neighbors_SecondClosestObjectNumber_Adjacent,Metadata_cmpd_target_type,Nuclei_PathName_SYTO,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_EulerNumber,Cells_ImageNumber,Cells_PathName_SYTO,Cytoplasm_FileName_HOECHST,Cells_FileName_HOECHST,Nuclei_FileName_SYTO,Cytoplasm_AreaShape_Area,Cytoplasm_Children_cytoplasm_Count,Cytoplasm_Metadata_AcqID,Nuclei_FileName_PHAandWGA,Cytoplasm_PathName_HOECHST,Cells_FileName_PHAandWGA,Cytoplasm_Children_nuclei_Count,Metadata_cmpd_image_id,Cytoplasm_FileName_SYTO,Cytoplasm_Location_Center_Z,Metadata_cmpd_pert_type,Cytoplasm_PathName_CONC,Cells_Parent_cells,Cytoplasm_ObjectNumber,Cells_FileName_SYTO,Cytoplasm_ImageNumber,Cytoplasm_AreaShape_ConvexArea,Metadata_cmpd_cp_id,Metadata_cmpd_pathway,Cytoplasm_FileName_MITO,Cytoplasm_FileName_PHAandWGA,Metadata_cmpd_onehot
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,str,i32,str,str,i32,str,str,i32,i32,i32,i32,str,str,str,str,i32,i32,i32,str,str,str,i32,i64,str,i32,str,str,i32,i32,str,i32,i32,i64,str,str,str,i64
"""PB000137_F08""","""PB000137""","""F08""",1.591848,0.020889,0.737056,-0.74838,0.609674,0.396334,11.070058,158.059344,0.071332,0.745752,1.353995,0.484623,0.885146,0.562251,0.0,-0.025716,0.646754,-1.222309,0.001439,-0.297185,-0.002363,0.005911,1.595439,0.804077,1.665911,1.511012,-0.04805,-0.007227,-6.420777,0.0,-137.158935,0.0,0.565479,-0.312143,…,"""/share/mikro2/nikon/spher-colo…",595,"""/share/mikro2/nikon/spher-colo…","""PB000137_F08""",4,"""Targeted""",null,677,5904,1,1,"""/share/mikro2/nikon/spher-colo…","""Well-F08-z6-HOECHST.ome.tiff""","""Well-F08-z6-HOECHST.ome.tiff""",null,3395,1,4185,null,"""/share/mikro2/nikon/spher-colo…","""Well-F08-z6-PHAandWGA.ome.tiff""",1,4185,"""Well-F08-z6-SYTO.ome.tiff""",0,"""trt""","""/share/mikro2/nikon/spher-colo…",1,1,"""Well-F08-z6-SYTO.ome.tiff""",1,3640,5532,"""Protein Tyrosine Kinase""","""Well-F08-z6-MITO.ome.tiff""","""Well-F08-z6-PHAandWGA.ome.tiff""",3
"""PB000139_E08""","""PB000139""","""E08""",0.163568,-0.372447,-0.817861,-0.702877,-0.831863,0.144748,35.820364,173.228011,-0.101273,1.780737,-0.002377,1.155035,-1.12823,-3.271169,0.0,

In [43]:
# columns_containing_metadata = [col_name for col_name in df.columns if 'Metadata_' in col_name]
# columns_containing_metadata
# df2=df.select([c for c in df.columns if not is_meta_column(c)])
df_floats=df.select([c for c in df.columns if not is_meta_column(c)]).select(float_columns)

# for c in df.columns:
#     if is_meta_column(c):
#         print(c)


In [30]:
# df2 = df.filter(pl.col('Metadata_cmpd_pert_type').is_in(cons))
# cons = ['pos_con']
nodmso = ['pos_con']
wdmso = ['neg_con']
df2 = df.filter(pl.col('Metadata_cmpd_target').is_in(cons))
df3 = df.filter(~pl.col('Metadata_cmpd_target').is_in(nodmso))
df4 = df.filter(pl.col('Metadata_cmpd_target').is_in(wdmso))

ColumnNotFoundError: unable to find column "Metadata_cmpd_target"; valid columns: ["Metadata_PlateWell", "Metadata_Barcode", "Metadata_Well", "Nuclei_RadialDistribution_ZernikeMagnitude_CONC_7_7", "Cells_RadialDistribution_ZernikePhase_MITO_4_2", "Nuclei_RadialDistribution_ZernikeMagnitude_PHAandWGA_9_1", "Cytoplasm_Correlation_Correlation_PHAandWGA_SYTO", "Cytoplasm_RadialDistribution_ZernikeMagnitude_HOECHST_4_4", "Nuclei_Location_MaxIntensity_X_MITO", "Cells_Granularity_13_MITO", "Cells_RadialDistribution_ZernikePhase_CONC_6_0", "Cytoplasm_RadialDistribution_ZernikePhase_MITO_2_2", "Nuclei_RadialDistribution_RadialCV_MITO_1of4", "Cells_RadialDistribution_ZernikeMagnitude_CONC_1_1", "Nuclei_Correlation_RWC_SYTO_HOECHST", "Nuclei_RadialDistribution_ZernikeMagnitude_SYTO_9_7", "Cells_Intensity_MaxIntensityEdge_SYTO", "Nuclei_Correlation_Manders_HOECHST_SYTO", "Cytoplasm_RadialDistribution_ZernikePhase_CONC_8_4", "Cytoplasm_Intensity_IntegratedIntensityEdge_PHAandWGA", "Nuclei_Correlati

In [31]:
df.head()

Metadata_PlateWell,Metadata_Barcode,Metadata_Well,Nuclei_RadialDistribution_ZernikeMagnitude_CONC_7_7,Cells_RadialDistribution_ZernikePhase_MITO_4_2,Nuclei_RadialDistribution_ZernikeMagnitude_PHAandWGA_9_1,Cytoplasm_Correlation_Correlation_PHAandWGA_SYTO,Cytoplasm_RadialDistribution_ZernikeMagnitude_HOECHST_4_4,Nuclei_Location_MaxIntensity_X_MITO,Cells_Granularity_13_MITO,Cells_RadialDistribution_ZernikePhase_CONC_6_0,Cytoplasm_RadialDistribution_ZernikePhase_MITO_2_2,Nuclei_RadialDistribution_RadialCV_MITO_1of4,Cells_RadialDistribution_ZernikeMagnitude_CONC_1_1,Nuclei_Correlation_RWC_SYTO_HOECHST,Nuclei_RadialDistribution_ZernikeMagnitude_SYTO_9_7,Cells_Intensity_MaxIntensityEdge_SYTO,Nuclei_Correlation_Manders_HOECHST_SYTO,Cytoplasm_RadialDistribution_ZernikePhase_CONC_8_4,Cytoplasm_Intensity_IntegratedIntensityEdge_PHAandWGA,Nuclei_Correlation_Overlap_CONC_MITO,Cytoplasm_RadialDistribution_ZernikePhase_CONC_8_2,Cells_AreaShape_EquivalentDiameter,Cells_RadialDistribution_ZernikePhase_CONC_5_1,Nuclei_RadialDistribution_ZernikePhase_HOECHST_9_3,Nuclei_RadialDistribution_ZernikeMagnitude_MITO_7_1,Nuclei_RadialDistribution_ZernikeMagnitude_PHAandWGA_9_3,Cells_RadialDistribution_ZernikeMagnitude_MITO_5_5,Cytoplasm_Granularity_3_HOECHST,Cells_RadialDistribution_ZernikePhase_MITO_9_9,Nuclei_RadialDistribution_ZernikePhase_SYTO_6_6,Nuclei_RadialDistribution_MeanFrac_MITO_2of4,Nuclei_Correlation_Manders_MITO_PHAandWGA,Cells_RadialDistribution_ZernikePhase_CONC_2_0,Cells_Correlation_Costes_CONC_PHAandWGA,Cytoplasm_RadialDistribution_ZernikeMagnitude_HOECHST_9_3,Nuclei_Correlation_RWC_MITO_HOECHST,…,Metadata_cmpd_cmpdname,Cytoplasm_PathName_SYTO,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_PathName_MITO,Metadata_cmpd_plate_well,Cytoplasm_Neighbors_SecondClosestObjectNumber_Adjacent,Metadata_cmpd_target_type,Nuclei_PathName_SYTO,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_EulerNumber,Cells_ImageNumber,Cells_PathName_SYTO,Cytoplasm_FileName_HOECHST,Cells_FileName_HOECHST,Nuclei_FileName_SYTO,Cytoplasm_AreaShape_Area,Cytoplasm_Children_cytoplasm_Count,Cytoplasm_Metadata_AcqID,Nuclei_FileName_PHAandWGA,Cytoplasm_PathName_HOECHST,Cells_FileName_PHAandWGA,Cytoplasm_Children_nuclei_Count,Metadata_cmpd_image_id,Cytoplasm_FileName_SYTO,Cytoplasm_Location_Center_Z,Metadata_cmpd_pert_type,Cytoplasm_PathName_CONC,Cells_Parent_cells,Cytoplasm_ObjectNumber,Cells_FileName_SYTO,Cytoplasm_ImageNumber,Cytoplasm_AreaShape_ConvexArea,Metadata_cmpd_cp_id,Metadata_cmpd_pathway,Cytoplasm_FileName_MITO,Cytoplasm_FileName_PHAandWGA
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,str,str,i32,str,str,i32,str,str,i32,i32,i32,i32,str,str,str,str,i32,i32,i32,str,str,str,i32,i64,str,i32,str,str,i32,i32,str,i32,i32,i64,i64,str,str
"""PB000137_F08""","""PB000137""","""F08""",1.591848,0.020889,0.737056,-0.74838,0.609674,0.396334,11.070058,158.059344,0.071332,0.745752,1.353995,0.484623,0.885146,0.562251,0.0,-0.025716,0.646754,-1.222309,0.001439,-0.297185,-0.002363,0.005911,1.595439,0.804077,1.665911,1.511012,-0.04805,-0.007227,-6.420777,0.0,-137.158935,0.0,0.565479,-0.312143,…,"""Gefitinib (ZD1839)""","""/share/mikro2/nikon/spher-colo…",595,"""/share/mikro2/nikon/spher-colo…","""PB000137_F08""",4,"""Targeted""",null,677,5904,1,1,"""/share/mikro2/nikon/spher-colo…","""Well-F08-z6-HOECHST.ome.tiff""","""Well-F08-z6-HOECHST.ome.tiff""",null,3395,1,4185,null,"""/share/mikro2/nikon/spher-colo…","""Well-F08-z6-PHAandWGA.ome.tiff""",1,4185,"""Well-F08-z6-SYTO.ome.tiff""",0,"""trt""","""/share/mikro2/nikon/spher-colo…",1,1,"""Well-F08-z6-SYTO.ome.tiff""",1,3640,5532,3,"""Well-F08-z6-MITO.ome.tiff""","""Well-F08-z6-PHAandWGA.ome.tiff"""
"""PB000139_E08""","""PB000139""","""E08""",0.163568,-0.372447,-0.817861,-0.702877,-0.831863,0.144748,35.820364,173.228011,-0.101273,1.780737,-0.002377,1.155035,-1.12823,-3.271169,0.0,-0.

In [ ]:
df2.shape[0]

In [ ]:
df3.shape[0]

In [ ]:
df4.shape[0]

In [ ]:
num_dmso = df2.filter(pl.col('Metadata_cmpd_moa')== 'dmso').shape[0]
num_dmso = int(num_dmso*0.1)
print(num_dmso)
dmso_df = df2.filter(pl.col('Metadata_cmpd_moa')=='dmso').sample(n=num_dmso, seed = 42)

df5 = df2.filter(~pl.col('Metadata_cmpd_moa').is_in(['dmso']))

df5 = pl.concat([df5, dmso_df])
# df_with_id = df.with_column(pl.arange(0, df.height).alias('unique_id'))
# mask = pl.col('Metadata_cmpd_pathway').is_not('dmso') | df.join(dmso_rows, on=list(df.columns), how='inner').with_column(pl.lit(True).alias('keep'))

# print(df5.shape)
cols = df5['Metadata_cmpd_moa'].unique().to_list()
for column in cols:
    # print(column)
    print(f"{column}:   {df5.filter(pl.col('Metadata_cmpd_moa')==column).shape[0]}")

In [ ]:
cols = df3['Metadata_cmpd_moa'].unique().to_list()
for column in cols:
    # print(column)
    print(f"{column}:   {df3.filter(pl.col('Metadata_cmpd_moa')==column).shape[0]}")

In [ ]:
# _ = makePCA(df5, name=filename)
# df2 = makeUMAP(df2, name=filename, is_supervised=False, use_pca=False)


In [ ]:
# _ = makeUMAP(df5, name=filename, nn=10, is_supervised=False, use_pca=False)

In [ ]:
# _ = makeUMAP(df5, name=filename,nn=10, is_supervised=True, use_pca=False)

In [ ]:
# _ = makePCA(df3, name=filename)

In [ ]:
#_ = makeUMAP(df3, name=filename, is_supervised=False, use_pca=True)

In [ ]:
#_ = makeUMAP(df3, name=filename, is_supervised=True, use_pca=True)

In [ ]:
float_columns=[pl.col(pl.Float32),pl.col(pl.Float64)]
tub_selected = ['Lexibulin', 'parbendazole', 'Nocodazole', 'ALBENDAZOLE', 'Colchicine', 'Vinorelbine ditartrate', 'Paclitaxel', 'plinabulin', 'MEBENDAZOLE', 'Vinblastine sulfate', 'Vincristine sulfate', 'Fenbendazole', 'ixabepilone']
cdk_selected = ['LY2857785', 'ON123300', 'DRF053-(R)', 'PHA-793887', 'Abemaciclib (Verzenio)', 'Roscovitine (Seliciclib, CYC202)', 'purvalanol-a', 'Palbociclib (Ibrance)', 'BMS-265246', 'Ribociclib (Kisqali)', 'TG-02', 'aminopurvalanol-a', 'RGB-286638']
parp_selected = ['E7449', 'Niraparib', 'Iniparib (BSI-201)', 'Rucaparib (phosphate)', 'IWR-1', 'AG14361', 'ME0328', 'EB-47', 'Veliparib', 'DR-2313', 'Talazoparib', 'AZD-2461', '(20S)-Protopanaxadiol']
akt_selected = ['Ipatasertib', 'triciribine', 'Buparlisib', 'Miransertib', 'AZD5363', 'MK-2206', 'PI-103', '10-DEBC', 'CCT128930', 'GSK690693', 'AR-12', 'GSK2110183', 'Uprosertib'] ## Kanske FPA-124
mapk_selected = ['RWJ-67657', 'Cobimetinib', 'Trametinib', 'Ulixertinib', 'EO-1428', 'SB-239063', 'SKF-86002', 'Binimetinib', 'TAK-715', 'PH-797804', 'SB-242235', 'LY2228820', 'Doramapimod']
hdac_selected = ['M344', 'Pimelic Diphenylamide 106', 'Abexinostat', 'Chidamide', 'Droxinostat', 'RG2833', 'Pyroxamide', 'Vorinostat', 'UF010', 'Resminostat (Hydrochloride)', 'Ricolinostat', 'SBHA', 'Tacedinaline', 'Scriptaid']

In [ ]:
is_super=False
nn = 50
targets = ['dmso','MAPK']
sel_cmpds = ['dmso'] + mapk_selected 
df_twoclass = df.filter(pl.col('Metadata_cmpd_moa_group').is_in(targets))
df_twoclass = df_twoclass.filter(pl.col('Metadata_cmpd_cmpdname').is_in(sel_cmpds))
embedded_df = makeUMAP(df_twoclass, name=filename, nn=nn, is_supervised=is_super, use_pca=False)



In [ ]:
# clusterer = hdb.HDBSCAN(
#     min_samples=2,
#     min_cluster_size=12,
# )
# embeddings = embedded_df.select(pl.col(['umap1', 'umap2']))
# embeddings = embeddings.to_numpy()
# clusterer.fit(embeddings)
# labels = clusterer.probabilities_
# labels_hard = clusterer.labels_
# embedded_df = embedded_df.with_columns([
#     pl.Series('class_prob', labels),  
#     pl.Series('class', labels_hard)   
#     ])

In [ ]:
# class sklearn.neighbors.NearestNeighbors(*, n_neighbors=5, radius=1.0, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, n_jobs=None)
k = 5  # Number of neighbors: tune this parameter based on your specific needs
knn = KNN(n_neighbors=k)

In [ ]:
targets = ['dmso','TUB']
sel_cmpds = ['dmso'] + tub_selected 
df_twoclass = df.filter(pl.col('Metadata_cmpd_moa_group').is_in(targets))
df_twoclass = df_twoclass.filter(pl.col('Metadata_cmpd_cmpdname').is_in(sel_cmpds))
_ = makeUMAP(df_twoclass, name=filename, nn=nn, is_supervised=is_super, use_pca=False)

In [ ]:
targets = ['dmso','HDAC']
sel_cmpds = ['dmso'] + hdac_selected 
df_twoclass = df.filter(pl.col('Metadata_cmpd_moa_group').is_in(targets))
df_twoclass = df_twoclass.filter(pl.col('Metadata_cmpd_cmpdname').is_in(sel_cmpds))
_ = makeUMAP(df_twoclass, name=filename, nn=nn, is_supervised=is_super, use_pca=False)

In [ ]:
targets = ['dmso','AKT']
sel_cmpds = ['dmso'] + akt_selected 
df_twoclass = df.filter(pl.col('Metadata_cmpd_moa_group').is_in(targets))
df_twoclass = df_twoclass.filter(pl.col('Metadata_cmpd_cmpdname').is_in(sel_cmpds))
_ = makeUMAP(df_twoclass, name=filename, nn=nn, is_supervised=is_super, use_pca=False)

In [ ]:
targets = ['dmso','PARP']
sel_cmpds = ['dmso'] + parp_selected 
df_twoclass = df.filter(pl.col('Metadata_cmpd_moa_group').is_in(targets))
df_twoclass = df_twoclass.filter(pl.col('Metadata_cmpd_cmpdname').is_in(sel_cmpds))
_ = makeUMAP(df_twoclass, name=filename, nn=nn, is_supervised=is_super, use_pca=False)

In [ ]:
targets = ['dmso','CDK']
sel_cmpds = ['dmso'] + cdk_selected 
df_twoclass = df.filter(pl.col('Metadata_cmpd_moa_group').is_in(targets))
df_twoclass = df_twoclass.filter(pl.col('Metadata_cmpd_cmpdname').is_in(sel_cmpds))
_ = makeUMAP(df_twoclass, name=filename, nn=nn, is_supervised=is_super, use_pca=False)

In [ ]:
# df['pc1'].max()

In [ ]:
# min_dist=0.2
# spread= 5
# n_epochs=None
# metric='cosine'
# for filename in filenames:
#     df = readData(filename)
#     _ = makePCA(df, name=filename)
#     # _ = makeUMAP(dataNpy, dataL, name=filename)
#     _ = makeUMAP(df, name=filename, nn=250)
#     _ = makeUMAP(df, name=filename, nn=300, is_supervised=False)


In [ ]:


# for statmet in statmets:
#     for filename in filenames:
#         dataNpy, dataL = readData(filename, statmet=statmet)
#         _ = makePCA(dataNpy, dataL, name=filename, statmet=statmet)
#         # _ = makeUMAP(dataNpy, dataL, name=filename, statmet=statmet)
#         _ = makeUMAP(dataNpy, dataL, name=filename, statmet=statmet, nn=150)
#         # _ = makeUMAP(dataNpy, dataL, name=filename, statmet=statmet, is_supervised=False)
#         _ = makeUMAP(dataNpy, dataL, name=filename, statmet=statmet, nn=150, is_supervised=False)

In [ ]:
# for filename in filenames:
#     dataNpy, dataL = readData(filename)
#     _ = makeMultiPlot(dataNpy, dataL, model_type='PCA', n_components=5)
#     _ = makeMultiPlot(dataNpy, dataL, model_type='UMAP', n_components=5)
#     _ = makeMultiPlot(dataNpy, dataL, model_type='UMAP', is_supervised=False, n_components=5)

In [ ]:

# for filename in filenames:
#     dataNpy, dataL = readData(filename, statmet='Mean')
#     _ = makeMultiPlot(dataNpy, dataL, model_type='PCA', statmet='Mean', n_components=5, nn=100)
#     _ = makeMultiPlot(dataNpy, dataL, model_type='UMAP', statmet='Mean', n_components=5, nn=100)
#     _ = makeMultiPlot(dataNpy, dataL, model_type='UMAP', statmet='Mean', is_supervised=False, n_components=5, nn=100)

In [ ]:
# for filename in filenames:
#     dataNpy, dataL = readData(filename, statmet='Median')
#     _ = makeMultiPlot(dataNpy, dataL, model_type='PCA', statmet='Median', n_components=5, nn=100)
#     _ = makeMultiPlot(dataNpy, dataL, model_type='UMAP', statmet='Median', n_components=5, nn=100)
#     _ = makeMultiPlot(dataNpy, dataL, model_type='UMAP', statmet='Median', is_supervised=False, n_components=5, nn=100)